

# **This code is ment to predict the average traffic speed in Kinshasa at different times of the day.**
> 1. It is meant to be executed on google colab
> 2. It generates an output file which consists of two columns, respectively the id/and target
> 3. It uses **lightgbm=== 3.3.3** as regressor,because of the instability of the latest lightgbm.not recognizing early_stoping_rounds as a parmeter.
> Therefore I start by uninstalling light gbm and then install the 3.3.3  version of it. remember to enter 'y' in an input box while uninstalling the latest version of Lightgbm

By **Bwenge840**

In [ ]:
!pip uninstall lightgbm

In [ ]:
!pip install lightgbm==3.3.3
import lightgbm
lightgbm.__version__

'3.3.3'

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import random
import os
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


import ast

from shapely.geometry import Point, LineString
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
import folium

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
path ='/content/gdrive/MyDrive/Yango DRC/'

train = pd.read_csv(os.path.join(path, 'Train.csv'))
test = pd.read_csv(os.path.join(path, 'Test.csv'))
graph_df = pd.read_csv(os.path.join(path, 'Graph.csv'))

# Preview train dataset
train.head()

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


,ID,persistent_id,day,prediction_type,count_norm_00_0_,count_norm_00_1_,count_norm_00_2_,count_norm_00_3_,count_norm_01_0_,count_norm_01_1_,count_norm_01_2_,count_norm_01_3_,count_norm_02_0_,count_norm_02_1_,count_norm_02_2_,count_norm_02_3_,count_norm_03_0_,count_norm_03_1_,count_norm_03_2_,count_norm_03_3_,count_norm_04_0_,count_norm_04_1_,count_norm_04_2_,count_norm_04_3_,count_norm_05_0_,count_norm_05_1_,count_norm_07_2_,count_norm_07_3_,count_norm_08_0_,count_norm_08_1_,count_norm_08_2_,count_norm_08_3_,count_norm_09_0_,count_norm_09_1_,count_norm_09_2_,count_norm_09_3_,count_norm_10_0_,count_norm_10_1_,count_norm_10_2_,count_norm_10_3_,count_norm_11_0_,count_norm_11_1_,count_norm_11_2_,count_norm_11_3_,count_norm_12_0_,count_norm_12_1_,count_norm_12_2_,count_norm_12_3_,count_norm_13_0_,count_norm_13_1_,count_norm_13_2_,count_norm_13_3_,count_norm_14_0_,count_norm_14_1_,count_norm_14_2_,count_norm_14_3_,count_norm_15_0_,count_norm_15_1_,count_norm_15_2_,count_norm_15_3_,count_norm_16_0_,count_norm_16_1_,count_norm_16_2_,count_norm_16_3_,count_norm_17_0_,count_norm_17_1_,count_norm_19_2_,count_norm_19_3_,count_norm_20_0_,count_norm_20_1_,count_norm_20_2_,count_norm_20_3_,count_norm_21_0_,count_norm_21_1_,count_norm_21_2_,count_norm_21_3_,count_norm_22_0_,count_norm_22_1_,count_norm_22_2_,count_norm_22_3_,count_norm_23_0_,count_norm_23_1_,count_norm_23_2_,count_norm_23_3_,speed_avg_00_0_,speed_avg_00_1_,speed_avg_00_2_,speed_avg_00_3_,speed_avg_01_0_,speed_avg_01_1_,speed_avg_01_2_,speed_avg_01_3_,speed_avg_02_0_,speed_avg_02_1_,speed_avg_02_2_,speed_avg_02_3_,speed_avg_03_0_,speed_avg_03_1_,speed_avg_03_2_,speed_avg_03_3_,speed_avg_04_0_,speed_avg_04_1_,speed_avg_04_2_,speed_avg_04_3_,speed_avg_05_0_,speed_avg_05_1_,speed_avg_07_2_,speed_avg_07_3_,speed_avg_08_0_,speed_avg_08_1_,speed_avg_08_2_,speed_avg_08_3_,speed_avg_09_0_,speed_avg_09_1_,speed_avg_09_2_,speed_avg_09_3_,speed_avg_10_0_,speed_avg_10_1_,speed_avg_10_2_,speed_avg_10_3_,speed_avg_11_0_,speed_avg_11_1_,speed_avg_11_2_,speed_avg_11_3_,speed_avg_12_0_,speed_avg_12_1_,speed_avg_12_2_,speed_avg_12_3_,speed_avg_13_0_,speed_avg_13_1_,speed_avg_13_2_,speed_avg_13_3_,speed_avg_14_0_,speed_avg_14_1_,speed_avg_14_2_,speed_avg_14_3_,speed_avg_15_0_,speed_avg_15_1_,speed_avg_15_2_,speed_avg_15_3_,speed_avg_16_0_,speed_avg_16_1_,speed_avg_16_2_,speed_avg_16_3_,speed_avg_17_0_,speed_avg_17_1_,speed_avg_19_2_,speed_avg_19_3_,speed_avg_20_0_,speed_avg_20_1_,speed_avg_20_2_,speed_avg_20_3_,speed_avg_21_0_,speed_avg_21_1_,speed_avg_21_2_,speed_avg_21_3_,speed_avg_22_0_,speed_avg_22_1_,speed_avg_22_2_,speed_avg_22_3_,speed_avg_23_0_,speed_avg_23_1_,speed_avg_23_2_,speed_avg_23_3_,target
0,12176140421885990288_X_first_holiday_X_morning_rush_hour,12176140421885990288,first_holiday,morning_rush_hour,0.00001,NaN,0.00004,0.00002,0.00013,0.00000,NaN,0.00005,0.00007,0.00004,NaN,0.00005,0.00006,0.00003,NaN,0.00003,0.00001,0.00004,0.00002,0.00002,0.00013,0.00007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.63094,NaN,11.71044,20.21771,22.29209,16.62020,NaN,10.27815,24.79203,20.46646,NaN,16.21162,22.28341,16.62296,NaN,22.49340,25.87176,16.89469,17.28203,23.29445,16.76556,15.96588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.99969
1,1822492832818209601_X_last_weekday_X_morning_rush_hour,1822492832818209601,last_weekday,morning_rush_hour,0.00004,0.00007,0.00001,0.00003,0.00005,NaN,NaN,NaN,NaN,NaN,0.00004,0.00006,0.00002,NaN,0.00000,NaN,0.00000,0.00006,0.00005,0.00014,0.00001,0.00009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
meaningfull_description=['persistent_id','struct_type','segments','edge_type','speed_limit','toponym','is_in_poor_condition','length','category']
merged_df = pd.merge(graph_df[meaningfull_description], train, on='persistent_id')
merged_tst = pd.merge(graph_df[meaningfull_description], test, on='persistent_id')
merged_df.shape,merged_tst.shape
to_encode=['toponym','is_in_poor_condition','edge_type']

In [ ]:
le=LabelEncoder()
for col in to_encode:
  merged_df[col]=le.fit_transform(merged_df[col])
  merged_tst[col]=le.fit_transform(merged_tst[col])


In [ ]:
merged_df.shape,merged_tst.shape

((26157, 173), (11322, 172))

In [ ]:
speed_columns=[
    'speed_avg_17_1_', 'speed_avg_17_0_', 'speed_avg_16_2_', 'speed_avg_15_3_', 'speed_avg_16_1_',
    'speed_avg_16_3_', 'speed_avg_20_0_', 'speed_avg_15_2_', 'speed_avg_19_2_', 'speed_avg_16_0_',
    'speed_avg_15_1_', 'speed_avg_20_1_', 'speed_avg_12_3_', 'speed_avg_14_2_', 'speed_avg_20_3_',
    'speed_avg_14_1_', 'speed_avg_15_0_', 'speed_avg_14_3_', 'speed_avg_19_3_', 'speed_avg_14_0_',
    'speed_avg_13_2_', 'speed_avg_12_1_', 'speed_avg_12_2_', 'speed_avg_13_1_', 'speed_avg_10_2_',
    'speed_avg_13_0_', 'speed_avg_10_3_', 'speed_avg_11_3_', 'speed_avg_11_1_', 'speed_avg_20_2_',
    'speed_avg_08_3_', 'speed_avg_10_1_', 'speed_avg_11_2_', 'speed_avg_09_1_', 'speed_avg_21_0_',
    'speed_avg_13_3_', 'speed_avg_11_0_', 'speed_avg_12_0_', 'speed_avg_09_2_', 'speed_avg_08_2_',
    'speed_avg_10_0_', 'speed_avg_21_1_', 'speed_avg_09_0_', 'speed_avg_09_3_', 'speed_avg_07_3_',
    'speed_avg_08_1_', 'speed_avg_21_2_', 'speed_avg_22_0_', 'speed_avg_08_0_', 'speed_avg_21_3_',
    'speed_avg_07_2_', 'speed_avg_22_1_', 'speed_avg_22_2_', 'speed_avg_22_3_', 'speed_avg_00_0_',
    'speed_avg_05_1_', 'speed_avg_05_0_', 'speed_avg_00_2_', 'speed_avg_04_3_', 'speed_avg_00_1_',
    'speed_avg_04_2_', 'speed_avg_00_3_', 'speed_avg_01_0_', 'speed_avg_01_2_', 'speed_avg_01_3_',
    'speed_avg_02_2_', 'speed_avg_02_0_', 'speed_avg_03_3_', 'speed_avg_03_2_', 'speed_avg_03_0_',
    'speed_avg_01_1_', 'speed_avg_02_3_', 'speed_avg_04_1_', 'speed_avg_02_1_', 'speed_avg_03_1_',
    'speed_avg_04_0_']

In [ ]:
for col in speed_columns:
      merged_df[f'{col}_max']=merged_df.groupby(['persistent_id'])[col].transform('max')
      merged_df[f'{col}_mean']=merged_df.groupby(['persistent_id'])[col].transform('mean')
      merged_df[f'{col}_median']=merged_df.groupby(['persistent_id'])[col].transform('median')

      merged_df[f'{col}segment_max']=merged_df.groupby(['prediction_type'])[col].transform('std')
      merged_df[f'{col}segment_min']=merged_df.groupby(['prediction_type'])[col].transform('var')
      merged_df[f'{col}segment_mean']=merged_df.groupby(['prediction_type'])[col].transform('mean')
      merged_df[f'{col}segment_median']=merged_df.groupby(['prediction_type'])[col].transform('median')

In [ ]:
for col in speed_columns:
      merged_tst[f'{col}_max']=merged_tst.groupby(['persistent_id'])[col].transform('max')
      merged_tst[f'{col}_mean']=merged_tst.groupby(['persistent_id'])[col].transform('mean')
      merged_tst[f'{col}_median']=merged_tst.groupby(['persistent_id'])[col].transform('median')


      merged_tst[f'{col}segment_max']=merged_tst.groupby(['prediction_type'])[col].transform('std')
      merged_tst[f'{col}segment_min']=merged_tst.groupby(['prediction_type'])[col].transform('var')
      merged_tst[f'{col}segment_mean']=merged_tst.groupby(['prediction_type'])[col].transform('mean')
      merged_tst[f'{col}segment_median']=merged_tst.groupby(['prediction_type'])[col].transform('median')

In [ ]:
interested_list=['length']

In [ ]:
def get_quartile_thresholds(column):
    min_value = column.min()
    max_value = column.max()
    quartiles = np.unique(np.linspace(min_value, max_value, num=4))
    return quartiles

# Create categorical features using quartile thresholds
for column in interested_list:
        quartiles = get_quartile_thresholds(merged_df[column])
        labels = ['low', 'medium', 'high']
        merged_df[column + '_category'] = pd.cut(merged_df[column], bins=quartiles, labels=labels, include_lowest=True)
        merged_df=pd.get_dummies(merged_df,columns=[column + '_category'])

for column in interested_list:
        quartiles = get_quartile_thresholds(merged_tst[column])
        labels = ['low', 'medium', 'high']
        merged_tst[column + '_category'] = pd.cut(merged_tst[column], bins=quartiles, labels=labels, include_lowest=True)
        merged_tst=pd.get_dummies(merged_tst,columns=[column + '_category'])


In [ ]:
column_to_encode=['day','prediction_type','struct_type']
train=pd.get_dummies(merged_df,columns=column_to_encode)
test=pd.get_dummies(merged_tst,columns=column_to_encode)
train.shape,test.shape

((26157, 716), (11322, 715))

In [ ]:
set(train.columns)-set(test.columns)

{'target'}

In [ ]:
train.shape,test.shape

((26157, 716), (11322, 715))

In [ ]:
SEED=12
# # Selecting the independent variables and the target variable
X = train.drop(['ID', 'persistent_id','target'], axis = 1).fillna(0)
y = train.target

In [ ]:
n_splits = 5

# Initialize the k-fold cross-validator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=12)

# Initialize an empty list to store the cross-validation results
rmses = []
fold_pred = []

# Define the LightGBM parameters
params = {
    'learning_rate': 0.05,
    'subsample': 1,
    'colsample_bytree': 0.7,
    'reg_lambda': 0.1,  # Add reg_lambda parameter for regularization
    'objective': 'regression',
    'metric': 'rmse',
    'random_seed': 12
}

# Convert the data to LightGBM Dataset format
train_data = lgb.Dataset(X, label=y)

# Loop through each fold
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f'Fold {fold+1}')

    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_val, y_val = X.iloc[val_index], y.iloc[val_index]

    # Convert the training and validation sets to LightGBM Dataset format
    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_val = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    # Train the model on the current fold
    model = lgb.train(params, lgb_train, num_boost_round=10000, valid_sets=[lgb_train, lgb_val],
                      early_stopping_rounds=200, verbose_eval=300)

    # Generate predictions on the validation set for the current fold
    preds_val = model.predict(X_val, num_iteration=model.best_iteration)

    # Calculate the RMSE for the current fold and store it in the list
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    print(f'Fold {fold+1} RMSE: {rmse:.4f}\n')
    rmses.append(rmse)

    # Generate predictions on the test set for the current fold
    p2 = model.predict(test[X.columns], num_iteration=model.best_iteration)
    fold_pred.append(p2)

# Calculate the mean RMSE across all folds
mean_rmse = np.mean(rmses)
print(f'Mean RMSE: {mean_rmse:.4f}\n')

Fold 1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.477321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97838
[LightGBM] [Info] Number of data points in the train set: 20925, number of used features: 704
[LightGBM] [Info] Start training from score 7.519072
Training until validation scores don't improve for 200 rounds
[300]	training's rmse: 0.97897	valid_1's rmse: 1.27371
[600]	training's rmse: 0.759976	valid_1's rmse: 1.20047
[900]	training's rmse: 0.620647	valid_1's rmse: 1.16639
[1200]	training's rmse: 0.520601	valid_1's rmse: 1.14377
[1500]	training's rmse: 0.442497	valid_1's rmse: 1.12937
[1800]	training's rmse: 0.378817	valid_1's rmse: 1.119
[2100]	training's rmse: 0.329374	valid_1's rmse: 1.11184
[2400]	training's rmse: 0.287713	valid_1's rmse: 1.10659
[2700]	training's rmse: 0.251855	valid_1's rmse: 1.10197
[3000]	training's rmse: 0.223885	valid_1's rmse: 1.09879
[3300]	training's rm

In [ ]:
preds = np.mean(fold_pred,axis=0)

In [ ]:
# # Create a submission fil
sub_file = pd.DataFrame({'ID': test.ID, 'target': preds})
sub_file.head(10)

,ID,target
0,15228565441713549875_X_first_weekday_X_evening_rush_hour,11.75633
1,15228565441713549875_X_first_holiday_X_evening_rush_hour,11.23993
2,15228565441713549875_X_first_holiday_X_morning_rush_hour,14.04725
3,15228565441713549875_X_lbo_weekday_X_evening_rush_hour,11.18670
4,15228565441713549875_X_other_holiday_X_morning_rush_hour,14.72499
5,15228565441713549875_X_other_holiday_X_evening_rush_hour,12.76954
6,15228565441713549875_X_last_weekday_X_morning_rush_hour,13.21174
7,15228565441713549875_X_other_weekday_X_morning_rush_hour,12.92965
8,15228565441713549875_X_last_weekday_X_evening_rush_hour,12.15083
9,15228565441713549875_X_other_weekday_X_evening_rush_hour,11.55223


In [ ]:
sub_file.to_csv('BaselineSubmission.csv', index = False) # Download subfile and submit to zindi for scoring

In [ ]:

# Get feature importances
feature_importance_df = pd.DataFrame(model.feature_importance(), columns=['importance'])
feature_importance_df['feature'] = X.columns

# Select top features
top_50_features = feature_importance_df.sort_values(by='importance', ascending=False).head(500)['feature'].tolist()


In [ ]:
feature_importance_df.sort_values(by='importance', ascending=False).head(700)

,importance,feature
87,1768,speed_avg_00_0_
108,1767,speed_avg_05_1_
7,1701,count_norm_00_0_
89,1667,speed_avg_00_2_
5,1663,length
28,1586,count_norm_05_1_
8,1574,count_norm_00_1_
88,1549,speed_avg_00_1_
90,1408,speed_avg_00_3_
10,1408,count_norm_00_3_
